# Pulse Sequence Evaluation

Will Kaufman, 2020

This notebook verifies the fidelity of pulse sequences identified by a reinforcement learning algorithm (as opposed to traditional techniques like average Hamiltonian theory).

In [ ]:
import spin_simulation as ss
import numpy as np
import pandas as pd
from datetime import datetime
#import plotly.express as px

In [ ]:
# import importlib
# importlib.reload(ss)
# importlib.reload(rlp)

# Initialize spin system

This sets the parameters of the system ($N$ spin-1/2 particles, which corresponds to a Hilbert space with dimension $2^N$). For the purposes of simulation, $\hbar \equiv 1$.

The total internal Hamiltonian is given by
$$
H_\text{int} = C H_\text{dip} + \Delta \sum_i^N I_z^{(i)}
$$
where $C$ is the coupling strength, $\Delta$ is the chemical shift strength (each spin is assumed to be identical), and $H_\text{dip}$ is given by
$$
H_\text{dip} = \sum_{i,j}^N d_{i,j} \left(3I_z^{(i)}I_z^{(j)} - \mathbf{I}^{(i)} \cdot \mathbf{I}^{(j)}\right)
$$

The WAHUHA pulse sequence is designed to remove the dipolar interaction term from the internal Hamiltonian. The pulse sequence is $\tau, P_{-x}, \tau, P_{y}, \tau, \tau, P_{-y}, \tau, P_{x}, \tau$.
The zeroth-order average Hamiltonian for the WAHUHA pulse sequence is
$$
H_\text{WHH} = \Delta / 3 \sum_i^N I_x^{(i)} + I_y^{(i)} + I_z^{(i)}
$$

In [ ]:
N = 4
dim = 2**N
coupling = 2*np.pi * 5e3    # coupling strength
delta = 2*np.pi * 500       # chemical shift strength (for identical spins)

(x,y,z) = (ss.x, ss.y, ss.z)
(X,Y,Z) = ss.get_total_spin(N, dim)

Ux = ss.get_rotation(X, np.pi/2)
Uxbar = ss.get_rotation(X, -np.pi/2)
Uy = ss.get_rotation(Y, np.pi/2)
Uybar = ss.get_rotation(Y, -np.pi/2)

In [ ]:
def evaluate_sequence(delay_samples, pulse_width_samples, Htarget,
                      seq, seq_type, eval_time, n=100, time_samples = 25):
    """Evaluate a candidate pulse sequence with a variety of delays and pulse
    widths.
    
    Arguments:
        delay_samples, pulse_width_samples: Numpy arrays of time values.
        seq: An array of indices to index the unitaries array for pulse sequence
            construction. Should be given in chronological order.
        eval_time: Time over which to evaluate the pulse sequence. If the pulse
            sequence is 10 microseconds, and eval_time is 1ms, then the sequence
            should be repeated 100 times and then the fidelity is measured.
    """
    fidelities = []
    delays = []
    pulse_widths = []
    types = []
    times = []
    reps = []
    for d in delay_samples:
        for p in pulse_width_samples:
            for _ in range(n):
                Hdip, Hint = ss.get_H(N, dim, coupling, delta)
                Ux = ss.get_propagator(Hint + X*np.pi/2/p, p)
                Uy = ss.get_propagator(Hint + Y*np.pi/2/p, p)
                Uxbar = ss.get_propagator(Hint - X*np.pi/2/p, p)
                Uybar = ss.get_propagator(Hint - Y*np.pi/2/p, p)
                Utau = ss.get_propagator(Hint, d)
                operators = [Ux, Uxbar, Uy, Uybar, Utau]
                operator_times = [p]*4 + [d]
                # calculate sequence propagator and duration
                Useq = np.eye(dim, dtype="complex128")
                seq_time = 0
                for s in seq:
                    Useq = operators[s] @ Useq
                    seq_time += operator_times[s]
                # calculate target propagator over sequence time
                Utarget = ss.get_propagator(Htarget, seq_time)
                
                # how many repetitions of the pulse sequence to perform for each observation
                num_reps = int(np.floor(eval_time / (time_samples * seq_time)))
                # how many observations to make to get to eval_time
                num_obs = int(np.ceil(eval_time/(num_reps * seq_time)))
                Useq_cum = np.eye(dim, dtype = "complex128")
                Utarget_cum = np.copy(Useq_cum)
                time = 0
                for o in range(num_obs):
                    fidelities.append(ss.fidelity(Useq_cum,
                                                  Utarget_cum))
                    delays.append(d)
                    pulse_widths.append(p)
                    types.append(seq_type)
                    times.append(time)
                    reps.append(o)
                    # update time and unitaries
                    time += seq_time * num_reps
                    Useq_cum = np.linalg.matrix_power(Useq, num_reps) @ Useq_cum
                    Utarget_cum = np.linalg.matrix_power(Utarget, num_reps) @ Utarget_cum
                    
    d = {'fidelity': fidelities, 'delay': delays, 
         'pulse_width': pulse_widths, 'type': types, 'time': times, 'rep': o * num_reps}
    df = pd.DataFrame(data=d)
    df['reward'] = -np.log10(1-df['fidelity'])
    return df

## Evaluate existing pulse sequences

In [ ]:
dfHoRD = evaluate_sequence(np.geomspace(1e-6,10e-6,2),
                           np.geomspace(.01e-6,1e-6,2), 
                           1/3 * delta * Z,
                           [4, 2, 4, 0, 0, 4, 1, 1, 2, 0, 4, 1, 2, 1, 4, 0, 2, 4],
                           "HoRD-qubit-5", eval_time = 1e-4, 100)

In [ ]:
dfHoRD.plot(x='p', y='reward', kind='scatter')

In [ ]:
df_WHH = evaluate_sequence(np.geomspace(1e-6,5e-6,3),
                           np.geomspace(.1e-6,1e-6,3), 
                           1/3 * delta * (X + Y + Z),
                           [4, 1, 4, 2, 4, 4, 3, 4, 0, 4],
                           "WHH-4 Sequence", 1e-4, 250)

In [ ]:
df_WHH.to_csv("WHH.csv")

## Evaluate candidate pulse sequences

Canditate pulse sequences:

Number | Max reward | Pulse sequence | Date
---|---|---|---
1| 4.90 | tau, xbar, tau, ybar, tau, x y | (6/6)
2| 7.12 | tau, xbar y, tau, xbar, tau, ybar xbar^2 | (6/6)
3| 7.30 | xbar, tau, xbar, tau, ybar x | (6/6)
4| 7.31 | x ybar xbar, tau, x ybar^2, tau, y, tau, xbar y^2 x^2 | (6/9)

I'll go in order and evaluate each one independently

## 1

tau, xbar, tau, ybar, tau, x y

In [ ]:
# # TODO define the propagators in code here...
# # run for lots of random dipolar interactions

# fidelities = []
# delays = []
# pulseWidths = []

# for delay in np.geomspace(1e-6,10e-6,5):
#     for pulseWidth in np.geomspace(.01e-6,1e-6,5):
#         for i in range(100):
#             Hdip, Hint = ss.get_H(N, dim, coupling, delta)
#             HWHH0 = ss.get_H_WHH_0(N, dim, delta)
#             UWHH0 = ss.get_propagator(HWHH0, 3*delay)
            
#             if pulseWidth > 0:
#                 Ux = ss.get_propagator(Hint + X*np.pi/2/pulseWidth, pulseWidth)
#                 Uy = ss.get_propagator(Hint + Y*np.pi/2/pulseWidth, pulseWidth)
#                 Uxbar = ss.get_propagator(Hint - X*np.pi/2/pulseWidth, pulseWidth)
#                 Uybar = ss.get_propagator(Hint - Y*np.pi/2/pulseWidth, pulseWidth)
#             Utau = ss.get_propagator(Hint, delay)

#             Useq = Uy @ Ux @ Utau @ Uybar @ Utau @ Uxbar @ Utau
#             fidelities.append(ss.fidelity(Useq, UWHH0))
#             delays.append(delay)
#             pulseWidths.append(pulseWidth)

In [ ]:
# # plot results
# d = {'fidelity': fidelities, 'delay': delays, 'pulseWidth': pulseWidths}
# df1 = pd.DataFrame(data=d)
# fig1 = px.scatter_3d(df1, x='delay', y='pulseWidth', z='fidelity', opacity=0.7)
# fig1.update_layout(scene={'xaxis': {'type': 'log'}, 'yaxis': {'type': 'log'}})
# fig1.show()

## 2

(right to left)
Uxbar @ Uxbar @ Uybar @ Utau @ Uxbar @ Utau @ Uxbar @ Uybar @ Utau

In [ ]:
df_2 = evaluate_sequence(np.geomspace(1e-6,10e-6,3),
                         np.geomspace(.01e-6,1e-6,3), 
                         1/3 * delta * (X + Y + Z),
                         [4, 3, 1, 4, 1, 4, 3, 1, 1],
                           "Candidate 2", 1e-4, 250)

In [ ]:
df_2.to_csv("candidate2.csv")

## 3

Utau @ Ux @ Uybar @ Utau @ Uy @ Utau @ Uxbar
<!-- xbar, tau, xbar, tau, ybar x -->

In [ ]:
df_3 = evaluate_sequence(np.geomspace(1e-6,10e-6,3),
                         np.geomspace(.01e-6,1e-6,3), 
                         1/3 * delta * (X + Y + Z),
                         [1, 4, 2, 4, 3, 0, 4],
                           "Candidate 3", 1e-4, 250)

In [ ]:
df_3.to_csv("candidate3.csv")

## 4

x ybar xbar, tau, x ybar^2, tau, y, tau, xbar y^2 x^2

In [ ]:
df_4 = evaluate_sequence(np.geomspace(1e-6,10e-6,3),
                         np.geomspace(.01e-6,1e-6,3), 
                         1/3 * delta * (X + Y + Z),
                         [0, 3, 1, 4, 0, 3, 3, 4, 2, 4, 1, 2, 2, 0, 0],
                           "Candidate 4", 1e-4, 250)

# Useq = Ux @ Ux @ Uy @ Uy @ Uxbar @ Utau @ Uy @ Utau @ Uybar @ Uybar @ Ux @ Utau @ Uxbar @ Uybar @ Ux

In [ ]:
df_4.to_csv("candidate4.csv")